## Getting Started

In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.geoenrichment import enrich
from arcgis.features import FeatureSet

In [2]:
gis = GIS(api_key="YOUR-API-KEY")

## Analysis Areas

In [3]:
df = pd.read_csv("data/cvs_pharmacies.csv")
df

,Business Name,Address,City,State,ZIP,NAICS Code,Brand Name
0,CVS,33499 Yucaipa Blvd,Yucaipa,CA,92399,446110,CVS
1,CVS,12280 Perris Blvd,Moreno Valley,CA,92557,446110,CVS
2,CVS,8148 E Santa Ana Canyon Rd,Anaheim,CA,92808,446110,CVS
3,CVS,120 E Bonita Ave,San Dimas,CA,91773,446110,CVS
4,CVS,4200 E 4th St,Ontario,CA,91764,446110,CVS
...,...,...,...,...,...,...,...
76,CVS,9840 Sierra Ave,Fontana,CA,92335,446110,CVS
77,CVS,5555 Philadelphia St,Chino,CA,91710,446110,CVS
78,CVS,16964 Slover Ave,Fontana,CA,92337,446110,CVS
79,CVS,1931 N Campus Ave,Upland,CA,91784,446110,CVS


## Enriching Areas

In [4]:
cvs_areas = [{"address":x, "areaType": "DriveTimeBuffer","bufferRadii":[5], "bufferUnits": "Minutes"} for x in df.to_dict('records')]

In [5]:
cvs_dt_5min = enrich(cvs_areas, analysis_variables=["AtRisk.SENIORS_CY"])
cvs_dt_5min.head(n=2)

,ID,OBJECTID,sourceCountry,X,Y,areaType,bufferUnits,bufferUnitsAlias,bufferRadii,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,SENIORS_CY,SHAPE
0,0,1,US,-117.073666,34.033263,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,3103,"{""rings"": [[[-117.04073818928451, 34.054948149..."
1,1,2,US,-117.224860,33.942061,DriveTimeBuffer,Minutes,Minutes,5,BlockApportionment:US.BlockGroups,2.191,2.576,1,4179,"{""rings"": [[[-117.23275220271273, 33.978367121..."


In [6]:
cvs_map = gis.map('Ontario, CA, USA', zoomlevel=10)
cvs_map.basemap = 'dark-gray'

cvs_map.add_layer(FeatureSet.from_dataframe(cvs_dt_5min), {'renderer':'ClassedColorRenderer','field_name':'SENIORS_CY', 'opacity':0.5})
cvs_map

MapView(layout=Layout(height='400px', width='100%'))

## Data Analysis

In [7]:
significant_senior_population = cvs_dt_5min[cvs_dt_5min['SENIORS_CY'] > 10000]
significant_senior_population.shape

(4, 15)

In [8]:
cvs_map2 = gis.map('Ontario, CA, USA', zoomlevel=10)
cvs_map2.basemap = 'dark-gray'

cvs_map2.add_layer(FeatureSet.from_dataframe(significant_senior_population.copy()), {'renderer':'ClassedColorRenderer','field_name':'SENIORS_CY', 'opacity':0.5})
cvs_map2

MapView(layout=Layout(height='400px', width='100%'))